## Lógica para obtención de datos con web scrapping

### Conexión a la página web

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from urllib.parse import quote

In [1]:
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--ignore-certificate-errors-spki-list")
# Inicializa el controlador de Selenium
driver = webdriver.Chrome(options=options)

# Navega a la página de El Palacio de Hierro
main_url = "https://www.elpalaciodehierro.com"

# Navega a la página de Mundo Hogar
driver.get(main_url + "/mundo-hogar.html")

# Espera implícitamente a que los elementos se carguen (puedes ajustar el tiempo según sea necesario)
driver.implicitly_wait(10)

### Extracción de urls de subpáginas

In [2]:
# Encontrar todos los elementos que contienen las subcategorías de hogar
subcategories_element = driver.find_elements(
    By.CLASS_NAME, "b-categories_navigation-item_1"
)

# Obtiene el HTML de las subcategorías (8 es Hogar)
subcategories_html = subcategories_element[8].get_attribute("innerHTML")

# Guarda en un diccionario las subcategorías b-categories_navigation-link_2 existentes
subcategories_link_2 = []
for subcategory in BeautifulSoup(subcategories_html).find_all(
    class_="b-categories_navigation-link_2"
):
    # Quitando el https://www.elpalaciodehierro.com
    # Si termina en .html no es una subcategoría
    if subcategory.get("href").endswith(".html"): continue
    subcategories_link_2.append(
        subcategory.get("href").replace(main_url, "")
    )
    print(subcategory.get("href").replace(main_url, ""))


/hogar/muebles/
/hogar/colchones/
/hogar/blancos/
/hogar/organizacion-limpieza/
/hogar/cocina/
/hogar/electrodomesticos/
/hogar/decoracion/
/hogar/bano/
/hogar/mesa-bar/
/hogar/clima-y-ventilacion/
/mochilas-y-maletas-de-viaje/
/libros/
/mascotas/
/outlet/hogar/
/temporada/navidad/decoracion-navidena/


### Extracción de urls anidados

In [3]:
# Guarda en un diccionario las subcategorías b-categories_navigation-link_3 existentes
subcategories_link_3 = []
for subcategory in BeautifulSoup(subcategories_html).find_all( 
    class_="b-categories_navigation-link_3"
):
    # quitandole el https://www.elpalaciodehierro.com
    subcategories_link_3.append(
        subcategory.get("href").replace(main_url, "")
    )
    print(subcategory.get("href").replace(main_url, ""))
    

/hogar/linea-blanca/refrigeradores/
/hogar/linea-blanca/lavadoras/
/hogar/linea-blanca/lavadoras/centros-de-lavado/
/hogar/linea-blanca/estufas-parrillas/
/hogar/linea-blanca/lavadoras/lavasecadoras/
/hogar/linea-blanca/cavas-de-vino/
/hogar/linea-blanca/hornos/
/hogar/linea-blanca/secadoras/
/hogar/linea-blanca/campanas-cocina/
/hogar/linea-blanca/lavavajillas/
/hogar/linea-blanca/frigobares/
/hogar/muebles/salas/
/hogar/muebles/recamaras/
/hogar/muebles/comedor/
/hogar/muebles/sillas-bancos/
/hogar/muebles/mesas/
/hogar/muebles/muebles-galerias/
/hogar/terraza-jardin/muebles/
/hogar/muebles/estantes-soportes-de-tv/
/hogar/colchones/colchones-king-size/
/hogar/colchones/colchones-matrimonial/
/hogar/colchones/boxes/
/hogar/colchones/colchones-queen-size/
/hogar/colchones/colchones-individual/
/hogar/blancos/edredones-duvets/
/hogar/blancos/sabanas/
/hogar/blancos/almohadas-protectores/
/hogar/blancos/cobertores-cobijas-y-frazadas/
/hogar/blancos/cojineria/
/hogar/cuarto-infantil/decor

### Diagrama jerárquico de urls

In [5]:
# Recorrrer el diccionario jerárquico
# Función para agregar una URL a la estructura jerárquica con tuberías
def agregar_url(diccionario, url):
    partes = url.strip('/').split('/')
    actual = diccionario

    for i, parte in enumerate(partes):
        es_ultimo = i == len(partes) - 1
        actual = actual.setdefault(parte, {"_ES_ULTIMO": es_ultimo})

# Función para imprimir la estructura jerárquica con tuberías y conexiones
def imprimir_estructura(diccionario, nivel=0):
    for clave, valor in diccionario.items():
        es_ultimo = valor.pop("_ES_ULTIMO", False)
        conector = "└──" if es_ultimo else "├──"
        print('│   ' * nivel + f'{conector} {clave}')
        imprimir_estructura(valor, nivel + 1)

# Crear un diccionario jerárquico vacío
estructura_jerarquica = {}

# Unir subcategories_link_2 y subcategories_link_3
categorias = subcategories_link_2 + subcategories_link_3

# Agregar cada URL a la estructura jerárquica
for url in categorias:
    agregar_url(estructura_jerarquica, url)

# Imprimir la estructura jerárquica con tuberías y conexiones
print("Palacio de Hierro")
imprimir_estructura(estructura_jerarquica)

Palacio de Hierro
├── hogar
│   └── muebles
│   │   └── salas
│   │   └── recamaras
│   │   └── comedor
│   │   └── sillas-bancos
│   │   └── mesas
│   │   └── muebles-galerias
│   │   └── estantes-soportes-de-tv
│   └── colchones
│   │   └── colchones-king-size
│   │   └── colchones-matrimonial
│   │   └── boxes
│   │   └── colchones-queen-size
│   │   └── colchones-individual
│   └── blancos
│   │   └── edredones-duvets
│   │   └── sabanas
│   │   └── almohadas-protectores
│   │   └── cobertores-cobijas-y-frazadas
│   │   └── cojineria
│   │   └── accesorios-de-cama
│   └── organizacion-limpieza
│   │   └── cestos-para-ropa
│   │   └── cocina
│   │   └── organizadores
│   │   └── closet
│   │   └── cajas-y-contenedores
│   │   └── escaleras
│   │   └── limpieza
│   │   └── estantes
│   └── cocina
│   │   └── baterias-cocina
│   │   └── cafe-te
│   │   └── utensilios-de-cocina
│   │   └── sartenes
│   │   └── ollas
│   │   └── reposteria
│   │   └── cuchillos
│   │   └── almacenamient

In [6]:
# Guardar la estructura jerárquica en un archivo JSON Sin poner el _ES_ULTIMO
def agregar_url2(diccionario, url):
    partes = url.strip('/').split('/')
    actual = diccionario

    for i, parte in enumerate(partes):
        es_ultimo = i == len(partes) - 1
        actual = actual.setdefault(parte, {})
        
# Guardar la estructura jerárquica en un archivo JSON
estructura_jerarquica_json = {}

for url in categorias:
    agregar_url2(estructura_jerarquica_json, url)
 
with open('estructura_jerarquica.json', 'w') as fp:
    json.dump(estructura_jerarquica_json, fp, indent=4)

In [7]:
# Obtener urls completas, /llave/valor/subvalor/...
def obtener_rutas_terminales(diccionario, ruta_actual="", separador="/"):
    rutas_terminales = []

    for clave, valor in diccionario.items():
        nueva_ruta = ruta_actual + separador + clave
        if not valor:  # Si no hay subcategorías, es una ruta terminal
            rutas_terminales.append(nueva_ruta)
        if isinstance(valor, dict) and valor:
            rutas_terminales.extend(obtener_rutas_terminales(valor, nueva_ruta))

    return rutas_terminales
# Obtener urls completas, /llave/valor/subvalor/...
urls = obtener_rutas_terminales(estructura_jerarquica)

# Imprimir urls completas
for url in urls:
    print(url)

/hogar/muebles/salas
/hogar/muebles/recamaras
/hogar/muebles/comedor
/hogar/muebles/sillas-bancos
/hogar/muebles/mesas
/hogar/muebles/muebles-galerias
/hogar/muebles/estantes-soportes-de-tv
/hogar/colchones/colchones-king-size
/hogar/colchones/colchones-matrimonial
/hogar/colchones/boxes
/hogar/colchones/colchones-queen-size
/hogar/colchones/colchones-individual
/hogar/blancos/edredones-duvets
/hogar/blancos/sabanas
/hogar/blancos/almohadas-protectores
/hogar/blancos/cobertores-cobijas-y-frazadas
/hogar/blancos/cojineria
/hogar/blancos/accesorios-de-cama
/hogar/organizacion-limpieza/cestos-para-ropa
/hogar/organizacion-limpieza/cocina
/hogar/organizacion-limpieza/organizadores
/hogar/organizacion-limpieza/closet
/hogar/organizacion-limpieza/cajas-y-contenedores
/hogar/organizacion-limpieza/escaleras
/hogar/organizacion-limpieza/limpieza
/hogar/organizacion-limpieza/estantes
/hogar/cocina/baterias-cocina
/hogar/cocina/cafe-te
/hogar/cocina/utensilios-de-cocina
/hogar/cocina/sartenes
/ho

### Flujo de web scraping

In [19]:
import json
import os
from urllib.parse import quote
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def obtener_datos_articulos(main_url, ruta, folder):
    # Escapar caracteres especiales
    ruta = quote(ruta)

    # Verifica si la carpeta de destino existe, y si no, la crea
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Inicializa el driver de Selenium (asegúrate de tener el controlador del navegador adecuado instalado)
    driver = webdriver.Chrome()

    try:
        # Abre la página web
        driver.get(main_url + ruta)

        # Espera a que la página se cargue completamente (puedes ajustar este tiempo según tus necesidades)
        wait = WebDriverWait(driver, 10)

        # Lista para almacenar los datos de los artículos
        datos_articulos = []

        while True:
            # Intenta cerrar el pop-up si aparece
            try:
                pop_up = driver.find_element(By.CLASS_NAME, 'clase-del-pop-up')  # Reemplaza con la clase del pop-up
                if pop_up.is_displayed():
                    pop_up.find_element(By.CLASS_NAME, 'clase-para-cerrar-pop-up').click()  # Reemplaza con la clase para cerrar el pop-up
            except:
                pass

            # Obtén el contenido HTML de la página
            html = driver.page_source

            # Parsea el HTML con BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Encuentra todos los artículos en la página
            articulos = soup.find_all('article', class_='l-plp-grid_item m-product')

            for articulo in articulos:
                # Verifica si el artículo tiene el contenido necesario
                if articulo.find('div', class_='b-product_tile-brand') is not None:
                    # Extrae el enlace del artículo
                    enlace_articulo = articulo.find('a', class_='b-product_tile-image')['href']

                    # Extrae la marca del artículo
                    marca = articulo.find('div', class_='b-product_tile-brand').text.strip()

                    # Extrae el nombre del artículo
                    nombre = articulo.find('h3', class_='b-product_tile-name').text.strip()

                    # Extrae la descripción del badge_promo
                    badge_promo = articulo.find('div', class_='b-product_tile-badge_promo').text.strip()

                    # Extrae la descripción del producto en sí, maneja el escenario en el que no existe descripción
                    descripcion_producto_element = articulo.find('meta', {'itemprop': 'description'})
                    descripcion_producto = descripcion_producto_element['content'].strip() if descripcion_producto_element else ''

                    # Extrae el precio anterior
                    precio_anterior_element = articulo.find('div', class_='b-product_price-old')
                    precio_anterior = precio_anterior_element.find('span', class_='b-product_price-value').text.strip() if precio_anterior_element else ''

                    # Extrae el precio actual
                    precio_actual_element = articulo.find('div', class_='b-product_price-sales')
                    precio_actual = precio_actual_element.find('span', class_='b-product_price-value').text.strip() if precio_actual_element else ''

                    # Extrae la lista de especificaciones adicionales
                    especificaciones = []
                    especificaciones_container = articulo.find('div', class_='b-product_tile-specifications')
                    if especificaciones_container:
                        for li in especificaciones_container.find_all('li'):
                            especificaciones.append(li.text.strip())

                    # Almacena los datos en un diccionario
                    datos_articulo = {
                        "Enlace": main_url + enlace_articulo,
                        "Marca": marca,
                        "Nombre": nombre,
                        "Badge Promocional": badge_promo,
                        "Descripción del Producto": descripcion_producto,
                        "Precio Anterior": precio_anterior,
                        "Precio Actual": precio_actual,
                    }

                    if especificaciones:
                        datos_articulo["Especificaciones"] = especificaciones

                    # Agrega los datos del artículo a la lista
                    datos_articulos.append(datos_articulo)

            # Verifica si hay una página siguiente
            siguiente_pagina = soup.find('li', class_='b-next-btn')
            if siguiente_pagina:
                # Obtén el enlace de la página siguiente
                enlace_siguiente = siguiente_pagina.find('a')['href']

                # Navega a la página siguiente
                driver.get(enlace_siguiente)

                # Espera un tiempo razonable para que la página siguiente cargue
                try:
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'b-pagination-elements')))
                except:
                    pass
            else:
                # No hay más páginas siguientes, sal del bucle
                break

    except Exception as e:
        return 1

    finally:
        # Cierra el navegador
        driver.quit()

    # Guarda los datos en un archivo JSON con el nombre de la ruta
    nombre_archivo = ruta.replace('/', '_').strip('_') + '.json'
    with open(f"{folder}/{nombre_archivo}", 'w', encoding='utf-8') as json_file:
        json.dump(datos_articulos, json_file, ensure_ascii=False, indent=4)

    print(f"Datos de los artículos guardados en '{nombre_archivo}'")
    
    return 0

# Cargar urls
urls = np.load('urls.npy', allow_pickle=True)

# Inicializa el driver de Selenium (asegúrate de tener el controlador del navegador adecuado instalado)
driver = webdriver.Chrome()

# Uso de la función
main_url = 'https://www.elpalaciodehierro.com'

# Para todas las rutas
folder = 'data'

# Para todas las rutas
for ruta in urls:
    intentos = 0
    while obtener_datos_articulos(main_url, ruta, folder) == 1:
        intentos += 1
        print(f"Intento N°{intentos} para datos de '{ruta}'")
driver.quit()

Datos de los artículos guardados en 'hogar_muebles_salas.json'
Datos de los artículos guardados en 'hogar_muebles_recamaras.json'
Datos de los artículos guardados en 'hogar_muebles_comedor.json'
Datos de los artículos guardados en 'hogar_muebles_sillas-bancos.json'
Datos de los artículos guardados en 'hogar_muebles_mesas.json'
Datos de los artículos guardados en 'hogar_muebles_muebles-galerias.json'
Datos de los artículos guardados en 'hogar_muebles_estantes-soportes-de-tv.json'
Datos de los artículos guardados en 'hogar_colchones_colchones-king-size.json'
Datos de los artículos guardados en 'hogar_colchones_colchones-matrimonial.json'
Datos de los artículos guardados en 'hogar_colchones_boxes.json'
Datos de los artículos guardados en 'hogar_colchones_colchones-queen-size.json'
Datos de los artículos guardados en 'hogar_colchones_colchones-individual.json'
Datos de los artículos guardados en 'hogar_blancos_edredones-duvets.json'
Datos de los artículos guardados en 'hogar_blancos_sabana